# LDA Topic Modeling with Gensim
https://wikidocs.net/24949 -> 전처리 및 LSA


In [50]:
brand = '2019kia'

In [51]:
# 논문용
import pandas as pd
file_cols = ['date', 'time', 'user', 'text']
file = pd.read_csv("./전처리/preprocessed/전처리최종_{}.csv".format(brand), encoding = 'utf-8', header = None, names = file_cols)
#file = pd.read_csv("./전처리/preprocessed/전처리최종_앱티브_{}.csv".format(brand), encoding = 'utf-8', header = None, names = file_cols)
file.head()

,date,time,user,text
0,2019-01-30,23:59:56,IAmBigHead901,nigga just sign name not read nothing
1,2019-01-30,23:59:13,Kvydvnise,now kia
2,2019-01-30,23:57:27,Chicago_Wolves,sure ? heat wave come game saturday . weather ...
3,2019-01-30,23:57:24,akemiutau_bot,kia really mature sound . like voice
4,2019-01-30,23:56:54,snikastyle,never understand used car lot open hood car . ...


In [3]:
#앱티브 : 10/1 까지만 다시 할당
file = file.loc[file['date'] != '2019-10-02']
file[:-1]

,date,time,user,text
0,2019-09-23,23:59:32,Netta_World,love u
1,2019-09-23,23:59:02,Miller_Geek,honestly windows bring back linux . every o is...
2,2019-09-23,23:58:57,4SideRell,isnt funny serious matter kia
3,2019-09-23,23:55:15,CowboyDBWest,nope . re one people tell truth garbage . saw ...
4,2019-09-23,23:55:00,SiSi_Simply,lmbooo kia ?
...,...,...,...,...
25495,2019-10-01,0:01:16,zervos19,kia sportage lx fwd silver suv door view detai...
25496,2019-10-01,0:00:32,NewsView100,job come america great but nowhere near great ...
25497,2019-10-01,0:00:17,Wayne75362249,impeachment proceeding important u . fail heal...
25498,2019-10-01,0:00:16,Azula163,bush not pay medicare d obama take debt also ....


In [52]:
#논문용
documents = file['text'].tolist()
print(len(documents))

23912


훈련에 사용할 뉴스 데이터의 개수는 11314여개임. 이 중 첫번째 훈련용 뉴스를 출력해본다.

In [53]:
#논문용
print(type(documents))
print(documents[0])

<class 'list'>
nigga just sign name not read nothing


뉴스 데이터에는 특수문자가 포함된 다수의 영어문장으로 구성되어져 있습니다. 이런 형식의 뉴스가 총 11,314개 존재합니다. 사이킷런이 제공하는 뉴스 데이터에서 target_name에는 본래 이 뉴스 데이터가 어떤 20개의 카테고리를 갖고있었는지가 저장되어져 있습니다. 이를 출력해보겠습니다.

In [54]:
#논문용
from nltk.tag import pos_tag

total_df = pd.DataFrame({'document': documents})
#특수문자 제거
total_df['clean_doc']= total_df['document'].str.replace("[^a-zA-Z]", " ").fillna("")
# 길이가 2이하인 단어는 제거 (길이가 짧은 단어 제거)
total_df['clean_doc'] = total_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
# 전체 단어에 대한 소문자 변환(이미 전처리 단계에서 완료)
#total_df['clean_doc'] = total_df['clean_doc'].apply(lambda x: x.lower())
#total_df['clean_doc'] = total_df['clean_doc'].str.replace("'ve", " have").str.replace("'s", " is").str.replace("'t", " not").str.replace("'m", " am").str.replace("'ll", " will")

In [55]:
print(total_df['clean_doc'])
print(type(total_df['clean_doc']))

0                    nigga just sign name not read nothing
1                                                  now kia
2        sure heat wave come game saturday weather peep...
3                       kia really mature sound like voice
4        never understand used car lot open hood car an...
                               ...                        
23907           new post electric kia niro sell month sale
23908                                           thank much
23909                            princess kia jahsjdj shut
23910    chicago auto show preview chevrolet kia subaru...
23911    kia dealer need take lesson lexus dealer custo...
Name: clean_doc, Length: 23912, dtype: object
<class 'pandas.core.series.Series'>


In [56]:
#논문용
#POS_DOC 열 생성: N과 J만 남김 or N과 J와 V만 남김
total_df['nj_doc'] = total_df['clean_doc']
total_df['njv_doc'] = total_df['clean_doc']
tagged_list = []
nj_list = []
njv_list = []
for i, doc in enumerate(total_df['clean_doc']):
    tagged_list = pos_tag(doc.split())
    nj_list = [t[0] for t in tagged_list if t[1].startswith('N') or t[1].startswith('J')]
    njv_list = [t[0] for t in tagged_list if t[1].startswith('N') or t[1].startswith('J') or t[1].startswith('V')]
    total_df['nj_doc'][i] = " ".join(nj_list)
    total_df['njv_doc'][i] = " ".join(njv_list)

데이터의 정제가 끝났습니다. 다시 첫번째 훈련용 뉴스만 출력하여 정제 전, 후에 어떤 차이가 있는지 확인해보겠습니다.

In [57]:
#논문용
total_df['clean_doc'][0]

'nigga just sign name not read nothing'

In [58]:
#논문용
total_df['clean_doc'][0]

'nigga just sign name not read nothing'

In [59]:
#논문용
total_df.head()

,document,clean_doc,nj_doc,njv_doc
0,nigga just sign name not read nothing,nigga just sign name not read nothing,name nothing,sign name read nothing
1,now kia,now kia,,kia
2,sure ? heat wave come game saturday . weather ...,sure heat wave come game saturday weather peep...,sure heat game saturday call whopping degree,sure heat wave come game saturday peep call wh...
3,kia really mature sound . like voice,kia really mature sound like voice,kia sound voice,kia mature sound voice
4,never understand used car lot open hood car . ...,never understand used car lot open hood car an...,used car lot open hood car anyone interested h...,understand used car lot open hood car anyone i...


우선 특수문자가 제거되었으며, if나 you와 같은 길이가 3이하인 단어가 제거된 것을 확인할 수 있습니다. 뿐만 아니라 대문자가 전부 소문자로 바뀌었습니다. 이제 뉴스 데이터에서 불용어를 제거합니다. 불용어를 제거하기 위해서 토큰화를 우선 수행합니다. 토큰화와 불용어 제거를 순차적으로 진행합니다. 

In [60]:
#논문용1
tokenized_doc = total_df['nj_doc'].apply(lambda x: x.split()) # 토큰화

In [61]:
#논문용2
tokenized2_doc = total_df['njv_doc'].apply(lambda x: x.split()) # 토큰화

In [62]:
#논문용3
tokenized3_doc = total_df['clean_doc'].apply(lambda x: x.split()) #토큰화

다시 첫번째 훈련용 뉴스를 출력합니다.

In [63]:
#논문용
print(tokenized_doc[0])
print(tokenized2_doc[0])
print(tokenized3_doc[0])

['name', 'nothing']
['sign', 'name', 'read', 'nothing']
['nigga', 'just', 'sign', 'name', 'not', 'read', 'nothing']


In [64]:
#논문용
tokenized_doc.head()

0                                      [name, nothing]
1                                                   []
2    [sure, heat, game, saturday, call, whopping, d...
3                                  [kia, sound, voice]
4    [used, car, lot, open, hood, car, anyone, inte...
Name: nj_doc, dtype: object

In [65]:
#논문용
tokenized2_doc.head()

0                          [sign, name, read, nothing]
1                                                [kia]
2    [sure, heat, wave, come, game, saturday, peep,...
3                          [kia, mature, sound, voice]
4    [understand, used, car, lot, open, hood, car, ...
Name: njv_doc, dtype: object

In [66]:
#논문용
tokenized3_doc.head()

0        [nigga, just, sign, name, not, read, nothing]
1                                           [now, kia]
2    [sure, heat, wave, come, game, saturday, weath...
3            [kia, really, mature, sound, like, voice]
4    [never, understand, used, car, lot, open, hood...
Name: clean_doc, dtype: object

기존에 있었던 불용어에 속하던 your, about, just, that, will, after 단어들이 사라졌을 뿐만 아니라, 토큰화가 수행된 것을 확인할 수 있습니다. 

# TF-IDF 행렬 만들기
불용어 제거를 위해 토큰화 작업을 수행하였지만, TfidfVectorizer(TF-IDF 챕터 참고)는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다. 그렇기 때문에 TfidfVectorizer를 사용해서 TF-IDF 행렬을 만들기 위해서 다시 토큰화 작업을 역으로 취소하는 작업을 수행해보도록 하겠습니다. 이를 역토큰화(Detokenization)라고 합니다. 

In [67]:
#논문용
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(total_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

total_df['nj_doc'] = detokenized_doc

In [68]:
#논문용
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(total_df)):
    t = ' '.join(tokenized2_doc[i])
    detokenized_doc.append(t)

total_df['njv_doc'] = detokenized_doc

In [69]:
#논문용
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(total_df)):
    t = ' '.join(tokenized3_doc[i])
    detokenized_doc.append(t)

total_df['clean_doc'] = detokenized_doc

역토큰화가 제대로 되었는지 다시 첫번째 훈련용 뉴스를 출력하여 확인해보겠습니다. 

In [70]:
#논문용
total_df['nj_doc'][0]

'name nothing'

In [71]:
#논문용
total_df['njv_doc'][0]

'sign name read nothing'

In [72]:
#논문용
total_df['clean_doc'][0]

'nigga just sign name not read nothing'

정상적으로 불용어가 제거된 상태에서 역토큰화가 수행되었음을 확인할 수 있습니다. 
이제 사이킷런의 TfidfVectorizer를 통해 단어 1,000개에 대한 TF-IDF 행렬을 만들 것입니다. 물론 텍스트 데이터에 있는 모든 단어를 가지고 행렬을 만들 수는 있겠지만, 여기서는 1,000개의 단어로 제한하도록 하겠습니다.

In [73]:
#논문용
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(total_df['nj_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(23912, 1000)

In [74]:
#논문용
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X2 = vectorizer.fit_transform(total_df['njv_doc'])
X2.shape # TF-IDF 행렬의 크기 확인

(23912, 1000)

In [75]:
#논문용
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X3 = vectorizer.fit_transform(total_df['clean_doc'])
X3.shape # TF-IDF 행렬의 크기 확인

(23912, 1000)

# bigram만들기
8. 단어 토큰화와 텍스트 클린업
각 문장의 구두점과 불필요한 문자를 제거하여, 단어의 list로 토크나이징합시다. 이작업에는 Gensim의 simple_preprocess() 가 훌륭합니다. 추가적으로 나는 구두점을 제거하기 위해서 deacc=True 로 설정하였습니다
http://www.engear.net/wp/topic-modeling-gensimpython/

In [31]:
'''
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True removes punctuations
 
data_words = list(sent_to_words(data))
 
print(data_words[:1])
'''

'\ndef sent_to_words(sentences):\n    for sentence in sentences:\n        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True removes punctuations\n \ndata_words = list(sent_to_words(data))\n \nprint(data_words[:1])\n'

9. Bigram과 Trigram 모델 만들기
Bigram은 문서에서 자주 발생되는 2연 단어입니다. Tigram은 자주 발생하는 3연 단어입니다. 이 예에서는 ‘front_bumper’,’oil_leak’,’maryland_college_park’등이 있습니다. Gensim의 Phrases 모델은 bigram, trigram, quadgram 등 그 이상을 구현할 수 있습니다. Phrases에 대한 두가지 중요한 인수는 min_count및 임계값입니다. 이 매개변수의 값이 높을수록 단어가 바이그램으로 결합되는 것이 어렵습니다.

In [32]:
'''
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
 
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
 
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])
'''

'\n# Build the bigram and trigram models\nbigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.\ntrigram = gensim.models.Phrases(bigram[data_words], threshold=100)\n \n# Faster way to get a sentence clubbed as a trigram/bigram\nbigram_mod = gensim.models.phrases.Phraser(bigram)\ntrigram_mod = gensim.models.phrases.Phraser(trigram)\n \n# See trigram example\nprint(trigram_mod[bigram_mod[data_words[0]]])\n'

# LSA 토픽모델링

이제 TF-IDF 행렬을 다수의 행렬로 분해해보도록 하겠습니다. 여기서는 사이킷런의 절단된 SVD(Truncated SVD)를 사용합니다. 절단된 SVD를 사용하면 차원을 축소할 수 있습니다. 원래 기존 뉴스 데이터가 20개의 뉴스 카테고리를 갖고있었기 때문에, 20개의 토픽을 가졌다고 가정하고 토픽 모델링을 시도해보겠습니다. 토픽의 숫자는 n_components의 파라미터로 지정이 가능합니다. 

여기서 svd_model.componets_는 앞서 배운 LSA에서 VT에 해당됩니다.

In [33]:
np.shape(svd_model.components_)

NameError: name 'np' is not defined

정확하게 토픽의 수 t × 단어의 수의 크기를 가지는 것을 볼 수 있습니다.

In [34]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

NameError: name 'svd_model' is not defined

각 20개의 행의 각 1,000개의 열 중 가장 값이 큰 5개의 값을 찾아서 단어로 출력합니다.

# LDA with Gensim

이제 각 단어에 정수 인코딩을 하는 동시에, 각 뉴스에서의 단어의 빈도수를 기록해보겠습니다. 여기서는 각 단어를 (word_id, word_frequency)의 형태로 바꾸고자 합니다. word_id는 단어가 정수 인코딩된 값이고, word_frequency는 해당 뉴스에서의 해당 단어의 빈도수를 의미합니다. 이는 gensim의 corpora.Dictionary()를 사용하여 손쉽게 구할 수 있습니다. 전체 뉴스에 대해서 정수 인코딩을 수행하고, 두번째 뉴스를 출력해봅시다. 

In [76]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for  text in tokenized_doc]
print(corpus[0]) #수행된 결과에서 첫번째 뉴스 출력. 첫번째 문서의 인덱스는 1

[(0, 1), (1, 1)]


In [77]:
from gensim import corpora
dictionary2 = corpora.Dictionary(tokenized2_doc)
corpus2 = [dictionary2.doc2bow(text) for  text in tokenized2_doc]
print(corpus2[0]) #수행된 결과에서 첫번째 뉴스 출력. 첫번째 문서의 인덱스는 1

[(0, 1), (1, 1), (2, 1), (3, 1)]


In [78]:
from gensim import corpora
dictionary3 = corpora.Dictionary(tokenized3_doc)
corpus3 = [dictionary2.doc2bow(text) for  text in tokenized3_doc]
print(corpus3[0]) #수행된 결과에서 첫번째 뉴스 출력. 첫번째 문서의 인덱스는 1

[(0, 1), (1, 1), (2, 1), (3, 1), (1856, 1)]


첫번째 뉴스의 출력 결과를 봅시다. 위의 출력 결과 중에서 (7, 1)는 정수 인코딩이 66으로 할당된 단어가 첫번째 뉴스에서는 한 번 등장하였음을 의미합니다. 7이라는 값을 가지는 단어가 정수 인코딩이 되기 전에는 어떤 단어였는지 확인하여봅시다. 이는 dictionary[]에 기존 단어가 무엇인지 알고자하는 정수값을 입력하여 확인할 수 있습니다.

In [79]:
print(dictionary[7]) #7번째 단어

sure


In [80]:
print(dictionary2[7]) #7번째 단어

degree


In [81]:
print(dictionary3[7]) #7번째 단어

kia


기존에는 단어 'faith'이었음을 알 수 있습니다. 총 학습된 단어의 개수를 확인해보겠습니다. 이는 dictionary의 길이를 확인하면 됩니다. 

In [82]:
len(dictionary)

23841

In [83]:
len(dictionary2)

25764

In [84]:
len(dictionary3)

26942

총 8,358개의 단어가 학습되었습니다. 이제 LDA 모델을 훈련시켜보겠습니다.

# LDA 모델 훈련시키기
기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있었으므로 토픽의 개수를 20으로 하여 LDA 모델을 학습시켜보도록 하겠습니다.

parameter 지정하기.
https://coredottoday.github.io/2018/09/17/%EB%AA%A8%EB%8D%B8-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D/

In [102]:
#NJV doc
import gensim
NUM_TOPICS = 3 #20개의 토픽, k=20
ldamodel2 = gensim.models.ldamodel.LdaModel(corpus2, num_topics = NUM_TOPICS, id2word=dictionary2, passes=15, iterations=1500)
ldamodel2.save('./Topicmodel/{}_NJV_{}_15_gensim_model.gensim'.format(brand, NUM_TOPICS))
topics2 = ldamodel2.print_topics(num_words=20)
for topic in topics2:
    print(topic)

(0, '0.087*"kia" + 0.024*"commercial" + 0.013*"get" + 0.012*"make" + 0.009*"superbowl" + 0.008*"good" + 0.008*"car" + 0.007*"see" + 0.007*"know" + 0.007*"people" + 0.007*"think" + 0.007*"want" + 0.006*"love" + 0.006*"say" + 0.005*"year" + 0.005*"super" + 0.005*"work" + 0.005*"great" + 0.005*"give" + 0.005*"time"')
(1, '0.072*"kia" + 0.012*"hai" + 0.010*"name" + 0.008*"tha" + 0.006*"blackpink" + 0.005*"day" + 0.004*"world" + 0.004*"hain" + 0.004*"thank" + 0.004*"tour" + 0.003*"aur" + 0.003*"area" + 0.003*"say" + 0.003*"bhi" + 0.003*"main" + 0.003*"come" + 0.003*"happy" + 0.003*"time" + 0.002*"manchester" + 0.002*"sir"')
(2, '0.098*"kia" + 0.021*"car" + 0.020*"new" + 0.014*"look" + 0.014*"sportage" + 0.012*"drive" + 0.011*"soul" + 0.008*"price" + 0.007*"take" + 0.006*"press" + 0.006*"auto" + 0.006*"hyundai" + 0.006*"telluride" + 0.006*"show" + 0.005*"get" + 0.005*"suv" + 0.005*"refresh" + 0.005*"vehicle" + 0.005*"debut" + 0.005*"check"')


In [103]:
#NJV doc
import gensim
NUM_TOPICS = 10 #20개의 토픽, k=20
ldamodel9 = gensim.models.ldamodel.LdaModel(corpus2, num_topics = NUM_TOPICS, id2word=dictionary2, passes=15, iterations=1500)
ldamodel9.save('./Topicmodel/{}_NJV_{}_15_gensim_model.gensim'.format(brand, NUM_TOPICS))
topics9 = ldamodel9.print_topics(num_words=20)
for topic in topics9:
    print(topic)

KeyboardInterrupt: 

In [50]:
#Clean doc
import gensim
NUM_TOPICS = 18 #20개의 토픽, k=20
ldamodel3 = gensim.models.ldamodel.LdaModel(corpus3, num_topics = NUM_TOPICS, id2word=dictionary3, passes=15, iterations=1500)
ldamodel3.save('clean_18_15_gensim_model.gensim')
topics3 = ldamodel3.print_topics(num_words=20)
for topic in topics3:
    print(topic)

(0, '0.050*"choose" + 0.033*"head" + 0.026*"replace" + 0.024*"just" + 0.022*"jason" + 0.019*"run" + 0.017*"drag" + 0.016*"remember" + 0.014*"ulsan" + 0.014*"coscentralauto" + 0.014*"clear" + 0.014*"ticket" + 0.012*"woman" + 0.012*"calculate" + 0.012*"veganism" + 0.011*"unfortunately" + 0.011*"many" + 0.010*"veganfood" + 0.010*"thank" + 0.009*"complaint"')
(1, '0.074*"choose" + 0.045*"america" + 0.041*"many" + 0.030*"employee" + 0.024*"county" + 0.021*"fan" + 0.020*"pvap" + 0.016*"hodge" + 0.014*"much" + 0.013*"office" + 0.012*"minor" + 0.012*"concern" + 0.011*"bullish" + 0.010*"chose" + 0.010*"superbowlcommercials" + 0.010*"mgf" + 0.010*"appear" + 0.009*"offensive" + 0.009*"meat" + 0.009*"sope"')
(2, '0.109*"choose" + 0.061*"concern" + 0.054*"know" + 0.016*"guhhhhh" + 0.016*"fit" + 0.015*"regulator" + 0.014*"venture" + 0.014*"broadway" + 0.012*"however" + 0.012*"iuinna" + 0.011*"don" + 0.011*"layoff" + 0.010*"feature" + 0.010*"life" + 0.009*"head" + 0.008*"capefear" + 0.008*"paqpaq" + 

# pass 갯수 구하기
http://www.engear.net/wp/topic-modeling-gensimpython/ 

In [48]:
import time
import warnings
import gensim
from gensim.models.coherencemodel import CoherenceModel

coherences=[]
perplexities=[]
passes=[]
warnings.filterwarnings('ignore')

for i in range(10):
    
    ntopics, nwords = 10, 20
    if i==0:
        p=1
    else:
        p=i*5
    tic = time.time()
    lda4 = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=ntopics, iterations=1500, passes=p)
    print('epoch',p,time.time() - tic)
    # tfidf, corpus 무슨 차이?
    # lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=ntopics, iterations=200000)

    cm = CoherenceModel(model=lda4, corpus=corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    print("Coherence",coherence)
    coherences.append(coherence)
    print('Perplexity: ', lda4.log_perplexity(corpus),'\n\n')
    perplexities.append(lda4.log_perplexity(corpus))

epoch 1 3.731863498687744
Coherence -9.307217523311566
Perplexity:  -8.385366060568076 


epoch 5 14.143043994903564
Coherence -9.44502498048043
Perplexity:  -8.054349140493292 


epoch 10 29.60848832130432
Coherence -8.617723308852892
Perplexity:  -8.044928884478006 


epoch 15 46.249239683151245
Coherence -8.971902951538556
Perplexity:  -7.9809728271906595 


epoch 20 58.115013122558594
Coherence -9.79474996175303
Perplexity:  -7.959344966593278 


epoch 25 73.51534748077393
Coherence -9.108365262972255
Perplexity:  -7.923483042112234 


epoch 30 87.76434254646301
Coherence -9.612262728338182
Perplexity:  -7.876465561015016 


epoch 35 96.17158651351929
Coherence -8.648830280642976
Perplexity:  -7.841626122944669 


epoch 40 114.56618475914001
Coherence -8.628443204123913
Perplexity:  -7.85100157988899 


epoch 45 128.52831387519836
Coherence -8.81643475013143
Perplexity:  -7.819234579729832 




In [46]:
from sklearn.model_selection import GridSearchCV
model = GridSearchCV(gensim.models.ldamodel.LdaModel, param_grid=search_params)
best_lda_model = model.best_estimator

NameError: name 'search_params' is not defined

# 최적 토픽 갯수 구하기

In [86]:
import warnings
import time
import gensim
from gensim.models.coherencemodel import CoherenceModel

coherencesNJV=[]
perplexitiesNJV=[]
passes=[]
warnings.filterwarnings('ignore')

for i in range(20):
    if i==0:
        ntopics = 1
    else:
        ntopics = (i+1)
    nwords = 100
    tic = time.time()
    lda5 = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary2, num_topics=ntopics, iterations=400, passes=15)
    print('ntopics',ntopics,time.time() - tic)

    #cm = CoherenceModel(model=lda5, corpus=corpus, coherence='u_mass')
    cm = CoherenceModel(model=lda5, texts = tokenized2_doc, dictionary = dictionary2, coherence='c_v')
    coherence = cm.get_coherence()
    print("Coherence: ",coherence)
    coherencesNJV.append(coherence)
    print('Perplexity: ', lda5.log_perplexity(corpus),'\n\n')
    perplexitiesNJV.append(lda5.log_perplexity(corpus))

ntopics 1 35.27652025222778
Coherence:  0.8136872565926516
Perplexity:  -8.172745002348243 


ntopics 2 86.49971866607666
Coherence:  0.7841100322944545
Perplexity:  -8.160607400923114 


ntopics 3 82.90421962738037
Coherence:  0.7775720725620836
Perplexity:  -8.178791548698399 


ntopics 4 77.5515079498291
Coherence:  0.7550291960376394
Perplexity:  -8.230395814848698 


ntopics 5 76.09523296356201
Coherence:  0.6768139837670305
Perplexity:  -8.36248162669458 


ntopics 6 73.94680452346802
Coherence:  0.7523717840044662
Perplexity:  -8.418719069084698 


ntopics 7 74.80311346054077
Coherence:  0.6874649155620319
Perplexity:  -8.46291430343537 


ntopics 8 71.74015069007874
Coherence:  0.7342381283264434
Perplexity:  -8.666189445678885 


ntopics 9 80.93096590042114
Coherence:  0.6899096303588732
Perplexity:  -8.915297184975033 


ntopics 10 79.07452440261841
Coherence:  0.705877535719794
Perplexity:  -9.261616632004369 


ntopics 11 78.94440269470215
Coherence:  0.6971786931707669
Per

In [87]:
print(coherencesNJV)
print(perplexitiesNJV)

[0.8136872565926516, 0.7841100322944545, 0.7775720725620836, 0.7550291960376394, 0.6768139837670305, 0.7523717840044662, 0.6874649155620319, 0.7342381283264434, 0.6899096303588732, 0.705877535719794, 0.6971786931707669, 0.6886051047019116, 0.6995583678914671, 0.6838351692479087, 0.6865091131117621, 0.6844223923450523, 0.6940046530893318, 0.6979002043352414, 0.6928695294545598, 0.6894569967706321]
[-8.172745002348243, -8.16060740043119, -8.178791892133857, -8.230396749047863, -8.362480311259532, -8.4187078881936, -8.462934495643976, -8.666209881474348, -8.915219538067085, -9.26167334282015, -9.652510712558467, -10.190439141679049, -10.74538073259766, -11.202032014087091, -11.688480172965523, -12.036439727662975, -12.295771601940352, -12.583124662115091, -12.86182676671815, -13.103863609606217]


# LDA 시각화 하기
LDA 시각화를 위해서는 pyLDAvis의 설치가 필요합니다. 윈도우의 명령 프롬프트나 MAC/UNIX의 터미널에서 아래의 명령을 수행하여 pyLDAvis를 설치하시기 바랍니다. 

pip install pyLDAvis
설치가 완료되었다면 LDA 시각화 실습을 진행합니다.

In [104]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel2, corpus2, dictionary2) #sort_topics = False
pyLDAvis.display(vis)

In [105]:
pyLDAvis.save_html(vis ,'./토픽모델결과/pyLDAvis_{}.html'.format(brand))

좌측의 원들은 각각의 20개의 토픽을 나타냅니다. 각 원과의 거리는 각 토픽들이 서로 얼마나 다른지를 보여줍니다. 만약 두 개의 원이 겹친다면, 이 두 개의 토픽은 유사한 토픽이라는 의미입니다. 위의 그림에서는 10번 토픽을 클릭하였고, 이에 따라 우측에는 10번 토픽에 대한 정보가 나타납니다. 한 가지 주의할 점은 LDA 모델의 출력 결과에서는 토픽 번호가 0부터 할당되어 0~19의 숫자가 사용된 것과는 달리 위의 LDA 시각화에서는 토픽의 번호가 1부터 시작하므로 각 토픽 번호는 이제 +1이 된 값인 1~20까지의 값을 가집니다. 

# 문서 별 토픽 분포 보기
위에서 토픽 별 단어 분포는 확인하였으나, 아직 문서 별 토픽 분포에 대해서는 확인하지 못 하였습니다. 우선 문서 별 토픽 분포를 확인하는 방법을 보겠습니다. 각 문서의 토픽 분포는 이미 훈련된 LDA 모델인 ldamodel[]에 전체 데이터가 정수 인코딩 된 결과를 넣은 후에 확인이 가능합니다. 여기서는 책의 지면의 한계로 상위 5개의 문서에 대해서만 토픽 분포를 확인해보겠습니다. 

In [106]:
for i, topic_list in enumerate(ldamodel2[corpus]):
    if i==10:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.44358692), (1, 0.44528162), (2, 0.111131474)]
1 번째 문서의 topic 비율은 [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
2 번째 문서의 topic 비율은 [(0, 0.90424377), (1, 0.047396574), (2, 0.048359636)]
3 번째 문서의 topic 비율은 [(0, 0.08512809), (1, 0.45860633), (2, 0.45626557)]
4 번째 문서의 topic 비율은 [(0, 0.8354236), (1, 0.032398656), (2, 0.1321777)]
5 번째 문서의 topic 비율은 [(0, 0.08725744), (1, 0.06739407), (2, 0.8453485)]
6 번째 문서의 topic 비율은 [(0, 0.18512416), (1, 0.6273152), (2, 0.18756065)]
7 번째 문서의 topic 비율은 [(0, 0.11117199), (1, 0.11117044), (2, 0.77765757)]
8 번째 문서의 topic 비율은 [(0, 0.4426782), (1, 0.11364022), (2, 0.44368157)]
9 번째 문서의 topic 비율은 [(0, 0.06667413), (1, 0.06714799), (2, 0.8661779)]


위의 출력 결과에서 (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미합니다. 예를 들어 0번째 문서의 토픽 비율에서 (7, 0.3050222)은 7번 토픽이 30%의 분포도를 가지는 것을 의미합니다. 위의 코드를 응용하여 좀 더 깔끔한 형태인 데이터프레임 형식으로 출력해보겠습니다.

In [107]:
def make_topictable_per_doc(model, corpus, texts):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(model[corpus]):
        doc = topic_list[0] if model.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [108]:
topictable = make_topictable_per_doc(ldamodel2, corpus2, tokenized2_doc)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,1.0,0.4643,"[(0, 0.46254268), (1, 0.464349), (2, 0.07310836)]"
1,1,2.0,0.5825,"[(0, 0.218196), (1, 0.19934238), (2, 0.5824616)]"
2,2,0.0,0.5925,"[(0, 0.5924966), (1, 0.19493365), (2, 0.212569..."
3,3,0.0,0.8617,"[(0, 0.86169815), (1, 0.06872418), (2, 0.06957..."
4,4,2.0,0.6007,"[(0, 0.3723287), (1, 0.026954835), (2, 0.60071..."
5,5,2.0,0.9021,"[(0, 0.04904117), (1, 0.048830584), (2, 0.9021..."
6,6,0.0,0.7504,"[(0, 0.7504), (1, 0.12561104), (2, 0.12398899)]"
7,7,0.0,0.7570,"[(0, 0.75697047), (1, 0.11956066), (2, 0.12346..."
8,8,2.0,0.5048,"[(0, 0.40657148), (1, 0.088614166), (2, 0.5048..."
9,9,1.0,0.8553,"[(0, 0.07211964), (1, 0.85530597), (2, 0.07257..."


In [109]:
len(topictable)

23912

In [110]:
merged = pd.merge(file, topictable, left_index=True, right_index=True)

In [111]:
merged.to_csv('./토픽모델결과/병합토픽테이블_{}.csv'.format(brand), encoding = 'utf-8')

In [112]:
topictable.to_csv('./토픽모델결과/토픽테이블_{}.csv'.format(brand), encoding = 'utf-8')

In [113]:
#필터링할 문서들의 문서번호 갖고오기
filter_doc = []
filtered_topic = 2.0
print(topictable.groupby('가장 비중이 높은 토픽').size())
#print(filter_doc)
print(type(topictable))
print(topictable['가장 비중이 높은 토픽'][3])
with open('./NJ_topic5_2번_0.5이상문서번호.txt', 'w') as f:
    for i in range(len(topictable)):
        if topictable['가장 비중이 높은 토픽'][i] ==2.0 :
            #filter_doc.append(topictable['문서 번호'][i])
            if topictable['가장 높은 토픽의 비중'][i] >= 0.5 :  #걸러야 할것. 2.0이 너무 확률이 큰 것.
                filter_doc.append(topictable['문서 번호'][i])
                f.write(str(topictable['문서 번호'][i])+"\n") #불러올 때 string = f.read()후, string.split("\n")할 것
#print(filter_doc)
print(len(filter_doc))

가장 비중이 높은 토픽
0.0    10453
1.0     6393
2.0     7066
dtype: int64
<class 'pandas.core.frame.DataFrame'>
0.0
6488
